Create neighborhood dataframe from Excel – neighborhood, lat, long

In [1]:
# Import pandas library using an alias
import pandas as pd
# library to handle data in a vectorized manner
import numpy as np

# library to handle JSON files
import json
# library to handle requests
import requests
# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize

# import geocoder
import geocoder
# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# map rendering library
import folium

In [2]:
df_chi_data = pd.read_excel('CCASF12010CMAP.xlsx')
df_chi_data.head()

,GEOGKEYX,GEOGNAME,LATITUDE,UNSIGNED LONGITUDE,LONGITUDE
0,GeogKey,Geog,NaN,NaN,NaN
1,1,Rogers Park,42.016667,87.666667,-87.666667
2,2,West Ridge,42.000000,87.683333,-87.683333
3,3,Uptown,41.966667,87.666667,-87.666667
4,4,Lincoln Square,41.966667,87.683333,-87.683333


In [3]:
df_chi_data.drop([0], axis = 0, inplace=True)
df_chi_data.drop(['UNSIGNED LONGITUDE'], axis=1, inplace=True)
df_chi_data.drop(['GEOGKEYX'], axis=1, inplace=True)
df_chi_data.head()

,GEOGNAME,LATITUDE,LONGITUDE
1,Rogers Park,42.016667,-87.666667
2,West Ridge,42.000000,-87.683333
3,Uptown,41.966667,-87.666667
4,Lincoln Square,41.966667,-87.683333
5,North Center,41.950000,-87.683333


Create empty venue df – neighborhood, venue name, venue id, venue lat, venue long, distance from neighborhood center, category, price

In [4]:
df_venues = pd.DataFrame(columns=['Neighborhood','VenueName','VenueID','Latitude','Longitude','Distance','Category','Price'])
df_venues

,Neighborhood,VenueName,VenueID,Latitude,Longitude,Distance,Category,Price


Loop through neighborhoods

search for venues by long/lat
use json to populate venue df

In [5]:
# define Foursquare variables
CLIENT_ID = 'CT3K4Z2AEBTWGOKQLQKZ135JJ3B44KOQTB4BMEJ4R0AXXWSD' # your Foursquare ID
CLIENT_SECRET = 'TSJKVMWFPKG2ZV4NYZQDVIR5FIYOSRICDTHHKHWSMV5JHDMZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [6]:
# create a function to repeat the same process to all the neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius, LIMIT, CATEGORY, INTENT):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}&intent={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            lat, 
            lng,
            VERSION,
            CATEGORY, 
            radius, 
            LIMIT, 
            INTENT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['venues']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            v['name'], 
            v['id'], 
            v['location']['lat'], 
            v['location']['lng'],  
            v['location']['distance'],  
            v['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Venue', 
                  'Venue ID', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Distance',
                  'Venue Category']
    
    return(nearby_venues)

In [7]:
chi_venues = getNearbyVenues(names=df_chi_data['GEOGNAME'],
                                   latitudes=df_chi_data['LATITUDE'],
                                   longitudes=df_chi_data['LONGITUDE'],
                                   radius=500,
                                   LIMIT=100, 
                                   CATEGORY = '4d4b7105d754a06374d81259',
                                   INTENT = 'browse'
                                  )
chi_venues.head()

,Neighborhood,Venue,Venue ID,Venue Latitude,Venue Longitude,Distance,Venue Category
0,Rogers Park,Charmers Cafe,5710dcf0498e87c71d20b69d,42.016164,-87.668250,142,Café
1,Rogers Park,Caribbean American Bakery,4b5dde6ef964a520fa7029e3,42.019371,-87.669705,392,Bakery
2,Rogers Park,Tjam Kitchen,5a2071ca47f876422319a3b6,42.019310,-87.666920,294,Restaurant
3,Rogers Park,Jarvis Grill,4c117c7e17002d7f4755e609,42.015989,-87.668880,198,Fast Food Restaurant
4,Rogers Park,Jamaican Bakery,51cf7f97498ee7d50a505393,42.018398,-87.669414,297,Bakery


In [8]:
chi_venues.shape

(1338, 7)

Remove chain restaurants (those with more than 5 locations)

In [9]:
chi_venues_grouped = chi_venues.groupby('Venue').filter(lambda x: len(x) > 5)
chi_venues_grouped

,Neighborhood,Venue,Venue ID,Venue Latitude,Venue Longitude,Distance,Venue Category
90,Uptown,Subway,4b5b3875f964a52031ec28e3,41.965282,-87.661418,461,Sandwich Place
121,Lincoln Square,Starbucks,4aa3dfaaf964a520384420e3,41.964799,-87.685861,294,Coffee Shop
128,Lincoln Square,Potbelly Sandwich Shop,49f4c21ff964a5204b6b1fe3,41.966985,-87.687272,327,Sandwich Place
143,Lincoln Square,Dunkin',4c52d9412543a593290bfc85,41.966271,-87.688664,443,Donut Shop
166,North Center,Starbucks,54273f56498e550c0584a8bb,41.947936,-87.688509,486,Coffee Shop
169,North Center,Potbelly Sandwich Shop,542eed7c498e15b89f63ad00,41.948428,-87.688678,475,Sandwich Place
179,Lake View,Dunkin',4b5b6256f964a520aef928e3,41.954298,-87.650165,478,Donut Shop
180,Lake View,Dunkin',532488dc498e89b38c11821c,41.947083,-87.653812,452,Donut Shop
185,Lake View,Subway,4bd4d3736798ef3bbf80628d,41.947220,-87.653939,449,Sandwich Place
202,Lake View,Subway,4a42e581f964a5205da61fe3,41.951745,-87.649436,199,Sandwich Place


In [62]:
chi_venues = chi_venues[chi_venues.Venue != "Dunkin'"]
chi_venues = chi_venues[chi_venues.Venue != "Potbelly Sandwich Shop"]
chi_venues = chi_venues[chi_venues.Venue != "Starbucks"]
chi_venues = chi_venues[chi_venues.Venue != "Subway"]
chi_venues = chi_venues[chi_venues.Venue != "Baskin-Robbins"]
chi_venues = chi_venues[chi_venues.Venue != "Popeyes Louisiana Kitchen"]
chi_venues = chi_venues[chi_venues.Venue != "McDonald's"]
chi_venues.shape

(1223, 7)

In [63]:
chi_venues_grouped = chi_venues.groupby('Venue').filter(lambda x: len(x) > 5)
chi_venues_grouped

,Neighborhood,Venue,Venue ID,Venue Latitude,Venue Longitude,Distance,Venue Category


Remove duplicates from venue df based on distance from neighborhood center (maybe sort by distance then keep first)

In [11]:
chi_venues.duplicated('Venue ID')
chi_venues_duplicates = chi_venues[chi_venues.duplicated(['Venue ID'])]
 
print("Duplicate venues are:", chi_venues_duplicates, sep='\n')

Duplicate venues are:
            Neighborhood                                          Venue  \
314           North Park                                   Coffee Joint   
315           North Park                                  Laschet's Inn   
317           North Park                                      Mod Pizza   
318           North Park                   Reclaimed Bar and Restaurant   
319           North Park                               Borinquen Lounge   
320           North Park                                   Pete's Pizza   
625   East Garfield Park                               Al's Under the L   
626   East Garfield Park                           Lake's Best Pizzaria   
627   East Garfield Park                                   Vegies Pizza   
628   East Garfield Park                            Supper Club Chicago   
629   East Garfield Park                               All Time Bar B Q   
630   East Garfield Park                                      C & D BBQ   
631

Sort venue list by venue ID and distance from neighborhood center

In [64]:
chi_venues.sort_values(by=['Venue ID', 'Distance'], inplace=True)
chi_venues.head()

,Neighborhood,Venue,Venue ID,Venue Latitude,Venue Longitude,Distance,Venue Category
695,The Loop,Atwood,3fd66200f964a520c7f01ee3,41.883205,-87.628191,426,New American Restaurant
221,Lincoln Park,Sai Cafe,3fd66200f964a520e1ed1ee3,41.918481,-87.653361,343,Sushi Restaurant
678,The Loop,Monk's Pub,40b28c80f964a52045fb1ee3,41.885640,-87.634339,269,Pub
127,Lincoln Square,Daily Bar & Grill,40b28c80f964a5205ffd1ee3,41.964823,-87.686073,305,Bar
163,North Center,Laschet's Inn,40b28c80f964a520a5fc1ee3,41.954091,-87.681978,469,German Restaurant


In [67]:
# Check entries for a given venue
chi_venues[chi_venues['Venue ID'] == '59b5338d28122f42d21e8656']

,Neighborhood,Venue,Venue ID,Venue Latitude,Venue Longitude,Distance,Venue Category
703,Near South Side,Woven & Bound - Marriott Marquis Chicago,59b5338d28122f42d21e8656,41.85331,-87.620071,464,American Restaurant
782,Douglas,Woven & Bound - Marriott Marquis Chicago,59b5338d28122f42d21e8656,41.85331,-87.620071,464,American Restaurant


In [68]:
# Drop duplicates, keeping the first entry only
chi_venues.drop_duplicates(subset="Venue ID", keep='first', inplace=True)
# Check our sample venue again
chi_venues[chi_venues['Venue ID'] == '59b5338d28122f42d21e8656']

,Neighborhood,Venue,Venue ID,Venue Latitude,Venue Longitude,Distance,Venue Category
703,Near South Side,Woven & Bound - Marriott Marquis Chicago,59b5338d28122f42d21e8656,41.85331,-87.620071,464,American Restaurant


In [69]:
# Check the number of entries
print(chi_venues.shape)

(1165, 7)


Reset index after removing rows

In [12]:
chi_venues = chi_venues.reset_index(drop=True)
chi_venues.head()

,Neighborhood,Venue,Venue ID,Venue Latitude,Venue Longitude,Distance,Venue Category
0,Rogers Park,Charmers Cafe,5710dcf0498e87c71d20b69d,42.016164,-87.668250,142,Café
1,Rogers Park,Caribbean American Bakery,4b5dde6ef964a520fa7029e3,42.019371,-87.669705,392,Bakery
2,Rogers Park,Tjam Kitchen,5a2071ca47f876422319a3b6,42.019310,-87.666920,294,Restaurant
3,Rogers Park,Jarvis Grill,4c117c7e17002d7f4755e609,42.015989,-87.668880,198,Fast Food Restaurant
4,Rogers Park,Jamaican Bakery,51cf7f97498ee7d50a505393,42.018398,-87.669414,297,Bakery


In [13]:
# how many venues for each neighborhood
chi_venues.groupby('Neighborhood').count()

,Venue,Venue ID,Venue Latitude,Venue Longitude,Distance,Venue Category
Neighborhood,,,,,,
Albany Park,48,48,48,48,48,48
Archer Heights,6,6,6,6,6,6
Armour Square,49,49,49,49,49,49
Ashburn,11,11,11,11,11,11
Auburn Gresham,14,14,14,14,14,14
Austin,15,15,15,15,15,15
Avalon Park,24,24,24,24,24,24
Avondale,15,15,15,15,15,15
Belmont Cragin,37,37,37,37,37,37


In [14]:
# how many unique categories
print('There are {} uniques categories.'.format(len(chi_venues['Venue Category'].unique())))

There are 106 uniques categories.


In [15]:
chi_venues.to_excel('ChicagoVenues.xlsx')

Get user ratings of a venue

Loop thru venue df

search users who liked a venue ??
use json to populate user df

In [16]:
# create a function to get users who liked each venue
def getVenueLikes(venueids):
    
    venues_list=[]
    for venue_id in venueids:
        #print(venue_id)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}'.format(
            venue_id, 
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
            
        # make the GET request
        results = requests.get(url).json()['response']['likes']
        
        # return price for each venue
        if "items" in results:
            for i in results['items']:
                venues_list.append([(venue_id, i['id'], 1) ])
        else:
            print("No likes for venue {}.".format(venue_id))

    venue_likes = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    venue_likes.columns = ['Venue ID', 'User ID', 'Rating']
    
    return(venue_likes)

In [17]:
chi_venue_likes = getVenueLikes(venueids=chi_venues['Venue ID'])
chi_venue_likes.head()

No likes for venue 51cf7f97498ee7d50a505393.
No likes for venue 501c2f2ee4b08947b4b93ad9.
No likes for venue 4f3235f519836c91c7c11167.
No likes for venue 55ad8b02498ee9a4d38f3437.
No likes for venue 4e16e1bdfa76a474496f5683.
No likes for venue 5765f851498ec7182331d0cc.
No likes for venue 53bb6e86498e01e11c501cb4.
No likes for venue 4f32480719836c91c7c83b66.
No likes for venue 5223916d11d27dd32e355ce5.
No likes for venue 50270532e4b0e6861f0acf73.
No likes for venue 56ae52ef498eed4822e4a4e4.
No likes for venue 4d8a63d04757721ee437b669.
No likes for venue 4f2741c3e4b067a32993f36f.
No likes for venue 55b220d4498ec1a2f280dc45.
No likes for venue 5b65327bcb3fd200391472e5.
No likes for venue 4b453428f964a520e10726e3.
No likes for venue 50f22f89e4b01e115acf4e98.
No likes for venue 4f44903019836ed00194e7fa.
No likes for venue 54a35873498e9be465ba3336.
No likes for venue 53d5b1d8498e5d7ed7628d9a.
No likes for venue 5554daea498e2644fc6f6f5c.
No likes for venue 4b8b1e8ff964a520c29332e3.
No likes f

No likes for venue 5907fcb86a5950378c1aced2.
No likes for venue 4c732ad89e706dcb01fb4c62.
No likes for venue 4f329f4519836c91c7e9b007.
No likes for venue 5a444a11a2c00b5c131097c7.
No likes for venue 52754a2711d2179816750de1.
No likes for venue 4bbfde2974a9a5932350cff6.
No likes for venue 51e1e439498eef8b3bbe33ba.
No likes for venue 4e1fa15245dd950bb0128dba.
No likes for venue 4e83e76b02d5a75503f44fa8.
No likes for venue 4fd39190e4b065401a9f0808.
No likes for venue 4f32ad3519836c91c7eeeb86.
No likes for venue 4c9e937154c8a1cd1b4e8f4b.
No likes for venue 4f44ea3619836ed001976f1f.
No likes for venue 574dcd8e498eeac0db3bd269.
No likes for venue 5a8dc64c5d891b280b22f689.
No likes for venue 4f322cc019836c91c7bd863e.
No likes for venue 4f44fa8519836ed00197e1df.
No likes for venue 4f32849d19836c91c7df997c.
No likes for venue 4f2a5e8ae4b00df25c994a61.
No likes for venue 4f32828719836c91c7decc39.
No likes for venue 4f4496ee19836ed001951a96.
No likes for venue 5086dc0ae4b0f0026eaa6fa2.
No likes f

No likes for venue 5166fa2be4b0b9e4740adc09.
No likes for venue 4f32611b19836c91c7d20744.
No likes for venue 4d7bdc3b34f7721e20c9ba25.
No likes for venue 54b422c6498ef4fbd3a99546.
No likes for venue 4d7c1e12da568cfa10cc5fff.
No likes for venue 4c903b376fbf224b733c528f.
No likes for venue 50e07d51e4b092e9a9889fd2.
No likes for venue 4f622cebe4b08e537e718768.
No likes for venue 5b3e2d95c4404e001c0af136.
No likes for venue 4f3282a519836c91c7ded860.
No likes for venue 4f0f4eb3e4b0e190ad5cbd93.
No likes for venue 4d93b1424f78f04d1770e8b9.
No likes for venue 4f806c2de4b0cd6f61a7eb69.
No likes for venue 4d93b1424f78f04d1770e8b9.
No likes for venue 4f806c2de4b0cd6f61a7eb69.
No likes for venue 5112f7fbe4b0138635017b01.
No likes for venue 4f322e0f19836c91c7be03e8.
No likes for venue 4f32934419836c91c7e52d5e.
No likes for venue 533c4c86498ee1398ca52db8.
No likes for venue 4f32026519833175d60951c5.
No likes for venue 4f3242c419836c91c7c617a6.
No likes for venue 4e81cc4c30f856c1923ca6f8.
No likes f

,Venue ID,User ID,Rating
0,5710dcf0498e87c71d20b69d,515900,1
1,5710dcf0498e87c71d20b69d,229698,1
2,5710dcf0498e87c71d20b69d,51646211,1
3,4b5dde6ef964a520fa7029e3,50475374,1
4,4b5dde6ef964a520fa7029e3,2967333,1


In [18]:
chi_venue_likes.shape

(1667, 3)

Use one-hot encoding on categories and load into new df (copy venue df first – 1he)

In [19]:
# one hot encoding
chi_onehot_cat = pd.get_dummies(chi_venues[['Venue Category']], prefix="", prefix_sep="")
chi_onehot_cat.head()

,Afghan Restaurant,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,...,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theme Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [20]:
chi_onehot_cat.shape

(1254, 106)

In [21]:
# add venue ID column back to dataframe
chi_onehot_cat['Venue ID'] = chi_venues['Venue ID']
chi_onehot_cat.head()

,Afghan Restaurant,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,...,Tapas Restaurant,Tea Room,Thai Restaurant,Theme Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wings Joint,Venue ID
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5710dcf0498e87c71d20b69d
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,4b5dde6ef964a520fa7029e3
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5a2071ca47f876422319a3b6
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4c117c7e17002d7f4755e609
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,51cf7f97498ee7d50a505393


In [22]:
chi_onehot_cat.shape

(1254, 107)

In [23]:
# move venue ID to the first column
fixed_columns = [chi_onehot_cat.columns[-1]] + list(chi_onehot_cat.columns[:-1])
chi_onehot_cat = chi_onehot_cat[fixed_columns]
chi_onehot_cat.head()

,Venue ID,Afghan Restaurant,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,...,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theme Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wings Joint
0,5710dcf0498e87c71d20b69d,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4b5dde6ef964a520fa7029e3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5a2071ca47f876422319a3b6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4c117c7e17002d7f4755e609,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,51cf7f97498ee7d50a505393,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
chi_onehot_cat.shape

(1254, 107)

Use one-hot encoding on neighborhoods

In [25]:
# one hot encoding
chi_onehot_nhood = pd.get_dummies(chi_venues[['Neighborhood']], prefix="", prefix_sep="")
chi_onehot_nhood.head()

,Albany Park,Archer Heights,Armour Square,Ashburn,Auburn Gresham,Austin,Avalon Park,Avondale,Belmont Cragin,Beverly,...,Washington Heights,Washington Park,West Elsdon,West Englewood,West Garfield Park,West Lawn,West Pullman,West Ridge,West Town,Woodlawn
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
# add venue ID column back to dataframe
chi_onehot_nhood['Venue ID'] = chi_venues['Venue ID']
# move venue ID to the first column
fixed_columns = [chi_onehot_nhood.columns[-1]] + list(chi_onehot_nhood.columns[:-1])
chi_onehot_nhood = chi_onehot_nhood[fixed_columns]

chi_onehot_nhood.head()

,Venue ID,Albany Park,Archer Heights,Armour Square,Ashburn,Auburn Gresham,Austin,Avalon Park,Avondale,Belmont Cragin,...,Washington Heights,Washington Park,West Elsdon,West Englewood,West Garfield Park,West Lawn,West Pullman,West Ridge,West Town,Woodlawn
0,5710dcf0498e87c71d20b69d,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4b5dde6ef964a520fa7029e3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5a2071ca47f876422319a3b6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4c117c7e17002d7f4755e609,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,51cf7f97498ee7d50a505393,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
chi_onehot_nhood.shape

(1254, 73)

Combine one-hot encoding dfs for categories and neighborhoods

In [28]:
chi_onehot_all = pd.merge(chi_onehot_nhood, chi_onehot_cat, on='Venue ID', how='outer')
chi_onehot_all.head()

,Venue ID,Albany Park,Archer Heights,Armour Square,Ashburn,Auburn Gresham,Austin,Avalon Park,Avondale,Belmont Cragin,...,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theme Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wings Joint
0,5710dcf0498e87c71d20b69d,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4b5dde6ef964a520fa7029e3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5a2071ca47f876422319a3b6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4c117c7e17002d7f4755e609,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,51cf7f97498ee7d50a505393,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
chi_onehot_all.shape

(1376, 179)

In [61]:
 # find users who have liked more than 3 venues
chi_venue_likes.groupby('User ID').filter(lambda x: len(x) > 5)

,Venue ID,User ID,Rating
13,561155b9498ec959cc2c09f1,38216300,1
55,4d28e38b8292236a88ca1dbb,38216300,1
64,4ede9e6ee5faee9991758cb2,18372438,1
68,4fdd0058e4b0ec5a8f166067,18372438,1
75,4b35afb9f964a5208a2e25e3,124547371,1
97,4ae23773f964a520ea8b21e3,124547371,1
161,56aad837498ebfb7f15d8c40,3462946,1
162,56aad837498ebfb7f15d8c40,18469680,1
167,5b7c5b461fa763002c85e395,589793,1
171,4a6ca0e5f964a5200ed11fe3,589793,1


Pick a test user.
Save test user ratings from user ratings to new df.

In [31]:
test_user_ratings = chi_venue_likes[chi_venue_likes['User ID'] == '30596207'].drop('User ID', 1)
test_user_ratings

,Venue ID,Rating
173,5786fed8498e6ad5339dc258,1
339,4c6899d9428a0f47867e001b,1
1628,4c8cedadb118b7131c921d01,1
1660,56f1adc5498eeb31c9ecfdef,1


In [32]:
#Resetting the index to avoid future issues
test_user_ratings = test_user_ratings.reset_index(drop=True)

In [33]:
test_user_ratings

,Venue ID,Rating
0,5786fed8498e6ad5339dc258,1
1,4c6899d9428a0f47867e001b,1
2,4c8cedadb118b7131c921d01,1
3,56f1adc5498eeb31c9ecfdef,1


Create new venue df of those the test user has rated (based on one-hot encoding df).

In [34]:
test_user_likes =  chi_onehot_all[chi_onehot_all['Venue ID'].isin(test_user_ratings['Venue ID'].tolist())]
test_user_likes

,Venue ID,Albany Park,Archer Heights,Armour Square,Ashburn,Auburn Gresham,Austin,Avalon Park,Avondale,Belmont Cragin,...,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theme Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wings Joint
135,5786fed8498e6ad5339dc258,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
218,4c6899d9428a0f47867e001b,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1360,4c8cedadb118b7131c921d01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1373,56f1adc5498eeb31c9ecfdef,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Drop unnecessary columns

In [35]:
#Resetting the index to avoid future issues
test_user_likes = test_user_likes.reset_index(drop=True)
#Dropping unnecessary issues due to save memory and to avoid issues
test_user_features = test_user_likes.drop('Venue ID', 1)
test_user_features

,Albany Park,Archer Heights,Armour Square,Ashburn,Auburn Gresham,Austin,Avalon Park,Avondale,Belmont Cragin,Beverly,...,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theme Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Create vector of test user’s ratings

In [36]:
test_user_ratings['Rating']

0    1
1    1
2    1
3    1
Name: Rating, dtype: int64

Use dot product of vector and test user’s ratings to build test user profile

In [37]:
#Dot produt to get weights
userProfile = test_user_features.transpose().dot(test_user_ratings['Rating'])
#The user profile
userProfile.shape

(178,)

Let's start by extracting the category table from the original dataframe:

In [38]:
#Now let's get the features of every restaurant in our original dataframe
features_df = chi_onehot_all.set_index(chi_onehot_all['Venue ID'])
features_df.head()

,Venue ID,Albany Park,Archer Heights,Armour Square,Ashburn,Auburn Gresham,Austin,Avalon Park,Avondale,Belmont Cragin,...,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theme Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wings Joint
Venue ID,,,,,,,,,,,,,,,,,,,,,
5710dcf0498e87c71d20b69d,5710dcf0498e87c71d20b69d,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4b5dde6ef964a520fa7029e3,4b5dde6ef964a520fa7029e3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5a2071ca47f876422319a3b6,5a2071ca47f876422319a3b6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4c117c7e17002d7f4755e609,4c117c7e17002d7f4755e609,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51cf7f97498ee7d50a505393,51cf7f97498ee7d50a505393,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Drop unnecessary columns from 1he df

In [39]:
#And drop the unnecessary information
features_df = features_df.drop('Venue ID', 1)
features_df.head()

,Albany Park,Archer Heights,Armour Square,Ashburn,Auburn Gresham,Austin,Avalon Park,Avondale,Belmont Cragin,Beverly,...,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theme Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wings Joint
Venue ID,,,,,,,,,,,,,,,,,,,,,
5710dcf0498e87c71d20b69d,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4b5dde6ef964a520fa7029e3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5a2071ca47f876422319a3b6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4c117c7e17002d7f4755e609,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51cf7f97498ee7d50a505393,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


With the input's profile and the complete list of venues and their categories/neighborhoods in hand, we're going to take the weighted average of every venue based on the input profile and recommend the top ten restuarants that most satisfy it.

In [40]:
#Multiply the features by the weights and then take the weighted average
recommendationTable_df = ((features_df*userProfile).sum(axis=1))/(userProfile.sum())
#Sort our recommendations in descending order
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
recommendationTable_df.head()

Venue ID
56f1adc5498eeb31c9ecfdef    0.500
50983e68f31c409be941bbe4    0.375
5786fed8498e6ad5339dc258    0.375
5388b07c498ea2dd8e2d676c    0.375
4c8cedadb118b7131c921d01    0.375
dtype: float64

In [41]:
#The final recommendation table
recommended_venues = chi_venues.loc[chi_venues['Venue ID'].isin(recommendationTable_df.head(5).keys())]
recommended_venues

,Neighborhood,Venue,Venue ID,Venue Latitude,Venue Longitude,Distance,Venue Category
135,Lincoln Square,La Boulangerie,5786fed8498e6ad5339dc258,41.965011,-87.678553,436,Bakery
214,Lincoln Park,Le Pain Quotidien,5388b07c498ea2dd8e2d676c,41.918250,-87.653660,350,Bakery
227,Lincoln Park,interurban cafe & pastry,50983e68f31c409be941bbe4,41.918481,-87.649186,212,Bakery
1238,Edgewater,Vincent,4c8cedadb118b7131c921d01,41.979804,-87.667818,404,New American Restaurant
1251,Edgewater,Defloured Bakery,56f1adc5498eeb31c9ecfdef,41.979801,-87.668015,408,Bakery


Add suggested ratings

In [42]:
recommended_ratings = pd.merge(recommended_venues, recommendationTable_df.rename('Rating'), left_on='Venue ID', right_index=True)
recommended_ratings

,Neighborhood,Venue,Venue ID,Venue Latitude,Venue Longitude,Distance,Venue Category,Rating
135,Lincoln Square,La Boulangerie,5786fed8498e6ad5339dc258,41.965011,-87.678553,436,Bakery,0.375
214,Lincoln Park,Le Pain Quotidien,5388b07c498ea2dd8e2d676c,41.918250,-87.653660,350,Bakery,0.375
227,Lincoln Park,interurban cafe & pastry,50983e68f31c409be941bbe4,41.918481,-87.649186,212,Bakery,0.375
1238,Edgewater,Vincent,4c8cedadb118b7131c921d01,41.979804,-87.667818,404,New American Restaurant,0.375
1251,Edgewater,Defloured Bakery,56f1adc5498eeb31c9ecfdef,41.979801,-87.668015,408,Bakery,0.500


Drop unnecessary columns

In [43]:
recommended_venues = recommended_ratings.drop('Venue ID', 1).drop('Distance', 1)
recommended_venues

,Neighborhood,Venue,Venue Latitude,Venue Longitude,Venue Category,Rating
135,Lincoln Square,La Boulangerie,41.965011,-87.678553,Bakery,0.375
214,Lincoln Park,Le Pain Quotidien,41.918250,-87.653660,Bakery,0.375
227,Lincoln Park,interurban cafe & pastry,41.918481,-87.649186,Bakery,0.375
1238,Edgewater,Vincent,41.979804,-87.667818,New American Restaurant,0.375
1251,Edgewater,Defloured Bakery,41.979801,-87.668015,Bakery,0.500


Plot the top 5

In [44]:
# get coordinates for Chicago
address = 'Chicago, IL'

geolocator = Nominatim(user_agent="chi_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chicago are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chicago are 41.8755616, -87.6244212.


In [45]:
# create map
recommendation_map = folium.Map(location=[latitude, longitude], zoom_start=11)

In [46]:
# add markers to the map
for lat, lon, name, cat, rat in zip(recommended_venues['Venue Latitude'], recommended_venues['Venue Longitude'], recommended_venues['Venue'], recommended_venues['Venue Category'], recommended_venues['Rating']):
    label = folium.Popup(str(name) + ' (' + str(cat) + ', ' + str(rat) + ')', parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(recommendation_map)
       
recommendation_map

Get recommendations for a different user

In [47]:
test_user_ratings = chi_venue_likes[chi_venue_likes['User ID'] == '26541006'].drop('User ID', 1)
test_user_ratings

,Venue ID,Rating
33,4b08e160f964a520171323e3,1
461,4df7f3f118a801cd9f119204,1
522,4aef2171f964a520ded521e3,1
614,593478c3123a1963f50fc66f,1
1665,4b0c18f9f964a520cf3623e3,1


In [48]:
#Resetting the index to avoid future issues
test_user_ratings = test_user_ratings.reset_index(drop=True)
test_user_ratings

,Venue ID,Rating
0,4b08e160f964a520171323e3,1
1,4df7f3f118a801cd9f119204,1
2,4aef2171f964a520ded521e3,1
3,593478c3123a1963f50fc66f,1
4,4b0c18f9f964a520cf3623e3,1


In [49]:
test_user_likes =  chi_onehot_all[chi_onehot_all['Venue ID'].isin(test_user_ratings['Venue ID'].tolist())]
test_user_likes

,Venue ID,Albany Park,Archer Heights,Armour Square,Ashburn,Auburn Gresham,Austin,Avalon Park,Avondale,Belmont Cragin,...,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theme Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wings Joint
30,4b08e160f964a520171323e3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
267,4df7f3f118a801cd9f119204,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
291,4aef2171f964a520ded521e3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
363,593478c3123a1963f50fc66f,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1375,4b0c18f9f964a520cf3623e3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
#Resetting the index to avoid future issues
test_user_likes = test_user_likes.reset_index(drop=True)
#Dropping unnecessary issues due to save memory and to avoid issues
test_user_features = test_user_likes.drop('Venue ID', 1)
test_user_features

,Albany Park,Archer Heights,Armour Square,Ashburn,Auburn Gresham,Austin,Avalon Park,Avondale,Belmont Cragin,Beverly,...,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theme Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
test_user_ratings['Rating']

0    1
1    1
2    1
3    1
4    1
Name: Rating, dtype: int64

In [52]:
#Dot produt to get weights
userProfile = test_user_features.transpose().dot(test_user_ratings['Rating'])
#The user profile
userProfile.shape

(178,)

In [53]:
#Now let's get the features of every restaurant in our original dataframe
features_df = chi_onehot_all.set_index(chi_onehot_all['Venue ID'])
features_df.head()

,Venue ID,Albany Park,Archer Heights,Armour Square,Ashburn,Auburn Gresham,Austin,Avalon Park,Avondale,Belmont Cragin,...,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theme Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wings Joint
Venue ID,,,,,,,,,,,,,,,,,,,,,
5710dcf0498e87c71d20b69d,5710dcf0498e87c71d20b69d,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4b5dde6ef964a520fa7029e3,4b5dde6ef964a520fa7029e3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5a2071ca47f876422319a3b6,5a2071ca47f876422319a3b6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4c117c7e17002d7f4755e609,4c117c7e17002d7f4755e609,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51cf7f97498ee7d50a505393,51cf7f97498ee7d50a505393,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
#And drop the unnecessary information
features_df = features_df.drop('Venue ID', 1)
features_df.head()

,Albany Park,Archer Heights,Armour Square,Ashburn,Auburn Gresham,Austin,Avalon Park,Avondale,Belmont Cragin,Beverly,...,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theme Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wings Joint
Venue ID,,,,,,,,,,,,,,,,,,,,,
5710dcf0498e87c71d20b69d,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4b5dde6ef964a520fa7029e3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5a2071ca47f876422319a3b6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4c117c7e17002d7f4755e609,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51cf7f97498ee7d50a505393,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
#Multiply the features by the weights and then take the weighted average
recommendationTable_df = ((features_df*userProfile).sum(axis=1))/(userProfile.sum())
#Sort our recommendations in descending order
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
recommendationTable_df.head()

Venue ID
4b59c45df964a520fa9628e3    0.4
56316c6a498ebdd0e0e529e4    0.4
4aff74bff964a520c43822e3    0.4
4e026a67e4cdbd9a51745782    0.4
593478c3123a1963f50fc66f    0.3
dtype: float64

In [56]:
#The final recommendation table
recommended_venues = chi_venues.loc[chi_venues['Venue ID'].isin(recommendationTable_df.head(5).keys())]
recommended_venues

,Neighborhood,Venue,Venue ID,Venue Latitude,Venue Longitude,Distance,Venue Category
232,Near North Side,Happy Camper,56316c6a498ebdd0e0e529e4,41.904306,-87.634370,486,Pizza Place
233,Near North Side,Lou Malnati's Pizzeria,4e026a67e4cdbd9a51745782,41.902591,-87.628703,479,Pizza Place
239,Near North Side,Domino's Pizza,4b59c45df964a520fa9628e3,41.903763,-87.633175,419,Pizza Place
263,Near North Side,Pizano's Pizza & Pasta,4aff74bff964a520c43822e3,41.898733,-87.628264,443,Pizza Place
351,Portage Park,Easy Street Pizza & Beer Garden,593478c3123a1963f50fc66f,41.949234,-87.766956,88,Pizza Place


In [57]:
recommended_ratings = pd.merge(recommended_venues, recommendationTable_df.rename('Rating'), left_on='Venue ID', right_index=True)
recommended_ratings

,Neighborhood,Venue,Venue ID,Venue Latitude,Venue Longitude,Distance,Venue Category,Rating
232,Near North Side,Happy Camper,56316c6a498ebdd0e0e529e4,41.904306,-87.634370,486,Pizza Place,0.4
233,Near North Side,Lou Malnati's Pizzeria,4e026a67e4cdbd9a51745782,41.902591,-87.628703,479,Pizza Place,0.4
239,Near North Side,Domino's Pizza,4b59c45df964a520fa9628e3,41.903763,-87.633175,419,Pizza Place,0.4
263,Near North Side,Pizano's Pizza & Pasta,4aff74bff964a520c43822e3,41.898733,-87.628264,443,Pizza Place,0.4
351,Portage Park,Easy Street Pizza & Beer Garden,593478c3123a1963f50fc66f,41.949234,-87.766956,88,Pizza Place,0.3


In [58]:
recommended_venues = recommended_ratings.drop('Venue ID', 1).drop('Distance', 1)
recommended_venues

,Neighborhood,Venue,Venue Latitude,Venue Longitude,Venue Category,Rating
232,Near North Side,Happy Camper,41.904306,-87.634370,Pizza Place,0.4
233,Near North Side,Lou Malnati's Pizzeria,41.902591,-87.628703,Pizza Place,0.4
239,Near North Side,Domino's Pizza,41.903763,-87.633175,Pizza Place,0.4
263,Near North Side,Pizano's Pizza & Pasta,41.898733,-87.628264,Pizza Place,0.4
351,Portage Park,Easy Street Pizza & Beer Garden,41.949234,-87.766956,Pizza Place,0.3


In [59]:
# create map
recommendation_map = folium.Map(location=[latitude, longitude], zoom_start=11)

In [60]:
# add markers to the map
for lat, lon, name, cat, rat in zip(recommended_venues['Venue Latitude'], recommended_venues['Venue Longitude'], recommended_venues['Venue'], recommended_venues['Venue Category'], recommended_venues['Rating']):
    label = folium.Popup(str(name) + ' (' + str(cat) + ', ' + str(rat) + ')', parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(recommendation_map)
       
recommendation_map